In [552]:
import pandas as pd
import numpy as np
pd.options.display.float_format = '{:,.2f}'.format
import seaborn as sns
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from math import sqrt
from datetime import date
date_to = str((date.today()))
import pygsheets
import configparser  
from sqlalchemy import create_engine, text

In [553]:
config = configparser.ConfigParser()  
config.read("/Users/Taisia1/Desktop/octacode/config.ini")  
username = config["dxcore"]["username"]
passwd = config["dxcore"]["passwd"]
host = config["dxcore"]["host"]
db = config["dxcore"]["db"]


def connect():
    engine = create_engine(f"postgresql://{username}:{passwd}@{host}/{db}")
    return engine
engine = connect()

In [554]:
req_orders = f"""select
    name as login, created_time
    from
        dxcore.dxcore.principals
    where
      principals.created_time >= '2023-03-15' and principals.created_time <= '{date_to}' 
    order by
        principals.created_time asc"""

with engine.connect() as conn:
    registrations = pd.DataFrame(conn.execute(text(req_orders)))   
registrations['created_time'] = pd.to_datetime(registrations['created_time'])
logins = registrations['login'].unique()
registrations

,login,created_time
0,up632605464,2023-03-15 07:34:13.505
1,up632605467,2023-03-15 07:59:57.309
2,up632605465,2023-03-15 07:59:57.447
3,up632605470,2023-03-15 07:59:58.116
4,up632605466,2023-03-15 07:59:58.220
...,...,...
5538,up632611662,2023-07-20 19:44:43.235
5539,up632611664,2023-07-20 19:59:56.900
5540,up632611665,2023-07-20 20:00:05.403
5541,up632611667,2023-07-20 21:33:07.207


In [555]:
req_orders = f"""
    select dxcore.principals.name as login, dxcore.principals.id, accounts.clearing_code, accounts.account_code
    from dxcore.dxcore.principals
    left join
        dxcore.dxcore.accounts as accounts on principals.id = accounts.owner_id 
    where
        name in {tuple(logins)}  
     """

with engine.connect() as conn:
    df_live = pd.DataFrame(conn.execute(text(req_orders)))   
df_live['symbols'] = df_live['account_code'].str.partition('_')[0]     
df_live

,login,id,clearing_code,account_code,symbols
0,up632605464,94840,LIVE,USDT_1_up632605464,USDT
1,up632605590,95045,LIVE,BTC_1_up632605590,BTC
2,up632605602,95058,None,None,None
3,up632605707,95250,None,None,None
4,up632605730,95295,LIVE,BTC_1_up632605730,BTC
...,...,...,...,...,...
6491,up632611569,117380,LIVE,USDT_1_up632611569,USDT
6492,up632611611,117445,None,None,None
6493,up632611646,117493,None,None,None
6494,up632611656,117502,None,None,None


In [556]:
username = config["accountmng"]["username"]
passwd = config["accountmng"]["passwd"]
host = config["accountmng"]["host"]
db = config["accountmng"]["db"]



def connect():
    engine = create_engine(f"postgresql://{username}:{passwd}@{host}/{db}")
    return engine
engine = connect()

In [557]:
req = f"""select
    login, user_id
    from
        accountmng.public.clients c
    where
       c.login in {tuple(logins)}
      """

with engine.connect() as conn:
    users = pd.DataFrame(conn.execute(text(req))) 
users['user_id'] = users['user_id'].astype(str)    
user_ids = users['user_id'].unique() 
users

,login,user_id
0,up632564248,cbb7557a-3407-4718-9b2a-d8424a2d43ea
1,up632605464,44c51907-be16-4166-b0f7-660d5fdf6805
2,up632605465,9c42e0df-2f12-4dc9-82e2-46a28b63e724
3,up632605466,42185294-7cb9-4912-a0bf-f1250fb11ea1
4,up632605467,ac649997-0846-46fb-b1dd-a430424c4aee
...,...,...
5533,up632611662,93cab09e-c04f-4394-8da1-ff29519e1157
5534,up632611664,8c1e0480-fc09-4ae8-953a-d823a3efd7df
5535,up632611665,d0362874-84a1-48d2-9339-29b447d77477
5536,up632611667,c5467578-ab51-48a1-b07b-6e7a0d87fc62


In [558]:
username = config["finance_control"]["username"]
passwd = config["finance_control"]["passwd"]
host = config["finance_control"]["host"]
db = config["finance_control"]["db"]


def connect():
    engine = create_engine(f"postgresql://{username}:{passwd}@{host}/{db}")
    return engine
engine = connect()

In [559]:
req = f"""SELECT auth_user_id as user_id, affiliate_partners.external_id as partner_id, affiliate_offers.external_id as offer_id, affiliate_offers.program_type
    FROM affiliate_users
    LEFT JOIN affiliate_partners ON affiliate_users.partner_id = affiliate_partners.id
    LEFT JOIN affiliate_offers ON affiliate_users.offer_id = affiliate_offers.id
    where 
       auth_user_id in {tuple(user_ids)}
    """

with engine.connect() as conn:
    partner_offer = pd.DataFrame(conn.execute(text(req)))
partner_offer['partner_id'] = partner_offer['partner_id'].astype('int64')
partner_offer['offer_id'] = partner_offer['offer_id'].astype('int64')
partner_offer

,user_id,partner_id,offer_id,program_type
0,001f608e-a6a5-4e06-b90b-f5f58bce0e5d,41,11,INTERNAL_PROMO
1,00264689-7863-4dff-bb03-45ee470154c1,21,11,INTERNAL_PROMO
2,002c8a3d-88e0-44ae-b167-da9b29635256,21,11,INTERNAL_PROMO
3,00319156-4b1a-4c88-acc6-de259f75076e,21,11,INTERNAL_PROMO
4,0079e804-7996-41b6-8f00-1caace29bb6b,29,6,CPA
...,...,...,...,...
1857,ff117d7d-2588-4199-b752-7cecdb7c5168,21,11,INTERNAL_PROMO
1858,ff2c5fb7-dfb6-4917-b050-21d20eca8cdc,21,11,INTERNAL_PROMO
1859,ff3c0f7f-23bd-4e43-a970-adaaacf357e0,42,11,INTERNAL_PROMO
1860,ff9424c4-9ccd-464a-9830-298414a75300,30,6,CPA


In [560]:
partner_offer = partner_offer.merge(users, how='left', on='user_id')

In [561]:
partner_offer['partner_id'].unique()

array([41, 21, 29, 30, 47, 43, 17, 42, 35,  5, 32, 27])

In [562]:
partner_offer.isna().sum()

user_id         0
partner_id      0
offer_id        0
program_type    0
login           0
dtype: int64

In [563]:
username = config["dxcore"]["username"]
passwd = config["dxcore"]["passwd"]
host = config["dxcore"]["host"]
db = config["dxcore"]["db"]


def connect():
    engine = create_engine(f"postgresql://{username}:{passwd}@{host}/{db}")
    return engine
engine = connect()

req = f"""
    SELECT account_code, activity_type, activities.created_time::DATE as transaction_time, principals.name as login, activities.description, activities.action_code,
    trim(trailing '$' FROM instruments.symbol) AS quote_currency, activity_legs.quantity as amount
    FROM dxcore.dxcore.activity_legs
    LEFT JOIN dxcore.dxcore.activities ON activities.id = activity_legs.activity_id
    LEFT JOIN dxcore.dxcore.accounts on accounts.id = activities.account_id
    LEFT JOIN dxcore.dxcore.principals on principals.id = accounts.owner_id
    LEFT JOIN dxcore.dxcore.instruments on instruments.id = activity_legs.instrument_id
    where activities.activity_type in ('DEPOSIT')
    and (activities.description not similar to ('%%(demo|Demo|test|Test|hedge|Hedge|COMP)%%') or activities.description is null)
    --and activities.action_code like '%%COMP%%'
    and accounts.clearing_code = 'LIVE'   
    and principals.name in {tuple(logins)}
    and activities.created_time >= '2023-03-15' and activities.created_time <= '{date_to}'
    ORDER BY activities.created_time desc
    """
  
with engine.connect() as conn:
    df_deposits = pd.DataFrame(conn.execute(text(req))).drop_duplicates()
df_deposits.columns = df_deposits.columns.str.replace('quote_currency', 'symbols')
df_deposits['transaction_time'] = pd.to_datetime(df_deposits['transaction_time'])
req_market_data = f"""
        select bid_time::TIMESTAMP::DATE as transaction_time, REPLACE(REPLACE(event_symbol, 'USD/', ''), '/USD', '') as symbols, --
        case
            when event_symbol in ('USD/JPY', 'USD/CNH', 'USD/MXN', 'USD/CAD', 'USD/CHF') then (1/bid_price)
            when event_symbol in ('BTC/USD', 'ETH/USD', 'EUR/USD', 'GBP/USD', 'AUD/USD', 'NZD/USD') then bid_price
        end as bid_price
    from dxcore.dxcore.quotes_history qh 
    where event_symbol in ('BTC/USD', 'ETH/USD', 'EUR/USD', 'GBP/USD', 'USD/JPY', 'USD/CNH', 
    						'USD/MXN', 'AUD/USD', 'USD/CAD', 'NZD/USD', 'USD/CHF')
    and bid_time >= '2018-12-20'
    --and bid_time::TIMESTAMP::DATE """

with engine.connect() as conn:
    df_market_data = pd.DataFrame(conn.execute(text(req_market_data))).drop_duplicates(subset=['transaction_time', 'symbols'])

df_market_data['transaction_time'] = pd.to_datetime(df_market_data['transaction_time']) 
df_deposits = (
    df_deposits.merge(df_market_data, how='left', on=['transaction_time', 'symbols'])
    .fillna({'bid_price': 1})
).drop_duplicates()
df_deposits

df_deposits['in_usd'] = df_deposits['amount'].astype('float') * df_deposits['bid_price'].astype('float')
df_deposits = df_deposits.groupby(['login']).agg({'in_usd': ['sum', 'mean', 'count']})
df_deposits.columns = ['deposit_sum', 'deposit_mean', 'deposit_count']
df_deposits 

,deposit_sum,deposit_mean,deposit_count
login,,,
up632605464,10.39,10.39,1
up632606116,100.00,100.00,1
up632608310,12.01,12.01,1
up632608311,5.27,5.27,1
up632608312,3.64,3.64,1
...,...,...,...
up632611041,46.89,46.89,1
up632611431,45.00,22.50,2
up632611441,20.64,20.64,1


In [564]:
df_deposits = df_deposits.reset_index()
df_deposits

,login,deposit_sum,deposit_mean,deposit_count
0,up632605464,10.39,10.39,1
1,up632606116,100.00,100.00,1
2,up632608310,12.01,12.01,1
3,up632608311,5.27,5.27,1
4,up632608312,3.64,3.64,1
...,...,...,...,...
61,up632611041,46.89,46.89,1
62,up632611431,45.00,22.50,2
63,up632611441,20.64,20.64,1
64,up632611457,20.00,20.00,1


In [565]:
partner_offer = partner_offer.merge(df_deposits, how='left', on='login')
partner_offer

,user_id,partner_id,offer_id,program_type,login,deposit_sum,deposit_mean,deposit_count
0,001f608e-a6a5-4e06-b90b-f5f58bce0e5d,41,11,INTERNAL_PROMO,up632610706,NaN,NaN,NaN
1,00264689-7863-4dff-bb03-45ee470154c1,21,11,INTERNAL_PROMO,up632608649,NaN,NaN,NaN
2,002c8a3d-88e0-44ae-b167-da9b29635256,21,11,INTERNAL_PROMO,up632608826,NaN,NaN,NaN
3,00319156-4b1a-4c88-acc6-de259f75076e,21,11,INTERNAL_PROMO,up632608598,NaN,NaN,NaN
4,0079e804-7996-41b6-8f00-1caace29bb6b,29,6,CPA,up632609897,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
1857,ff117d7d-2588-4199-b752-7cecdb7c5168,21,11,INTERNAL_PROMO,up632610438,NaN,NaN,NaN
1858,ff2c5fb7-dfb6-4917-b050-21d20eca8cdc,21,11,INTERNAL_PROMO,up632610378,NaN,NaN,NaN
1859,ff3c0f7f-23bd-4e43-a970-adaaacf357e0,42,11,INTERNAL_PROMO,up632608830,NaN,NaN,NaN
1860,ff9424c4-9ccd-464a-9830-298414a75300,30,6,CPA,up632609406,NaN,NaN,NaN


In [566]:
offers = pd.read_csv('/Users/Taisia1/Desktop/octacode/partner_offer/offers.csv')
partners = pd.read_csv('/Users/Taisia1/Desktop/octacode/partner_offer/partners.csv')
offers

,offer_id,offer,offer_description
0,6,6 CPA_1,Основная CPA программа.
1,7,7 REVSHARE_1,Основная revshare программа.
2,11,11 CROSS PROMO,Кросс-промо Bitcoin Industry
3,12,12 CEX:IO,клиенты от CEX:IO
4,14,14 CPL,"CPL программа для Мета (fb, ig)"


In [567]:
partners

,partner_id,partner,partner_description
0,17,17 Masha test program,Тестовая партнерка Маши. Используется для тест...
1,27,27 tiktok post CryptoMalaysia 04/23,Тестовый пост в тикток https://vt.tiktok.com/Z...
2,29,29 Google ad 05/23,Тестовая реклама в Google
3,30,30 tm Crypto Portal 04/23,Промо через телеграм канал Crypto Portal https...
4,35,35 Meta MP,"Партнер Михаил, льет трафик с Мета (facebook, ..."
5,21,21 Content@mybro,Инфлюенсер ТГ
6,41,41 TGpartner,Инфлюенсер ТГ (crypto miami)
7,42,12312345bk,Инфлюенсер ТГ (crypto mountains)
8,43,btcindustry,Инфлюенсер в тг (bitcoin industry)
9,47,crypto,Инфлюенсер ТГ (crypto miami)


In [568]:
df = registrations.merge(df_live, how='left', on='login').merge(partner_offer, how='left', on='login')
df

,login,created_time,id,clearing_code,account_code,symbols,user_id,partner_id,offer_id,program_type,deposit_sum,deposit_mean,deposit_count
0,up632605464,2023-03-15 07:34:13.505,94840,LIVE,USDT_1_up632605464,USDT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,up632605467,2023-03-15 07:59:57.309,94842,LIVE,BTC_1_up632605467,BTC,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,up632605465,2023-03-15 07:59:57.447,94843,LIVE,BTC_1_up632605465,BTC,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,up632605470,2023-03-15 07:59:58.116,94844,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,up632605466,2023-03-15 07:59:58.220,94845,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6491,up632611662,2023-07-20 19:44:43.235,117512,None,None,None,93cab09e-c04f-4394-8da1-ff29519e1157,47.00,11.00,INTERNAL_PROMO,NaN,NaN,NaN
6492,up632611664,2023-07-20 19:59:56.900,117513,None,None,None,8c1e0480-fc09-4ae8-953a-d823a3efd7df,47.00,11.00,INTERNAL_PROMO,NaN,NaN,NaN
6493,up632611665,2023-07-20 20:00:05.403,117514,LIVE,USDT_1_up632611665,USDT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6494,up632611667,2023-07-20 21:33:07.207,117516,LIVE,USDT_1_up632611667,USDT,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [569]:
df

,login,created_time,id,clearing_code,account_code,symbols,user_id,partner_id,offer_id,program_type,deposit_sum,deposit_mean,deposit_count
0,up632605464,2023-03-15 07:34:13.505,94840,LIVE,USDT_1_up632605464,USDT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,up632605467,2023-03-15 07:59:57.309,94842,LIVE,BTC_1_up632605467,BTC,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,up632605465,2023-03-15 07:59:57.447,94843,LIVE,BTC_1_up632605465,BTC,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,up632605470,2023-03-15 07:59:58.116,94844,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,up632605466,2023-03-15 07:59:58.220,94845,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6491,up632611662,2023-07-20 19:44:43.235,117512,None,None,None,93cab09e-c04f-4394-8da1-ff29519e1157,47.00,11.00,INTERNAL_PROMO,NaN,NaN,NaN
6492,up632611664,2023-07-20 19:59:56.900,117513,None,None,None,8c1e0480-fc09-4ae8-953a-d823a3efd7df,47.00,11.00,INTERNAL_PROMO,NaN,NaN,NaN
6493,up632611665,2023-07-20 20:00:05.403,117514,LIVE,USDT_1_up632611665,USDT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6494,up632611667,2023-07-20 21:33:07.207,117516,LIVE,USDT_1_up632611667,USDT,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [570]:
username = config["dxcore"]["username"]
passwd = config["dxcore"]["passwd"]
host = config["dxcore"]["host"]
db = config["dxcore"]["db"]

def connect():
    engine = create_engine(f"postgresql://{username}:{passwd}@{host}/{db}")
    return engine
engine = connect()

req_orders = f"""select
    principals.name as login,
    accounts.account_code as account_id,
    order_instrument.symbol as order_symbol,
    split_part(order_instrument.symbol, '/', 2) as symbol,
    order_instrument.additional_fields::json->-0->'val' as pair_type,
    activities.order_id as order_id, orders.created_time, orders.parameters,
    coalesce(order_legs.position_code, activity_legs.position_code, activities.linked_position_code, '') as position_code,
    abs(activity_legs.quantity) as quantity, activities.transaction_time::DATE,
    order_legs.price as price,
    coalesce(orders.extensions::json->0->'val'->'PL_SETTLED_IN_TRADE_CURRENCY', '0')::text::decimal as PNL_3,
    coalesce(orders.extensions::json->1->'val'->'PL_SETTLED_IN_TRADE_CURRENCY', '0')::text::decimal as PNL_1,
    coalesce(orders.extensions::json->2->'val'->'PL_SETTLED_IN_TRADE_CURRENCY', '0')::text::decimal as PNL_2,
    coalesce(case when coalesce(orders.parameters::json->>'ORDER_EXEC_STRATEGY_NAME', opening_order.parameters::json->>'ORDER_EXEC_STRATEGY_NAME', 'B_BOOK') = 'FX_STP' 
    then round(abs((activity_legs.price - hedge.price) * orders.filled_quantity), 5) end, 0) as markup,
    coalesce(order_legs.position_effect, '') as position_effect
    from
        dxcore.dxcore.activities as activities
    inner join
        dxcore.dxcore.accounts as accounts on activities.account_id = accounts.id and accounts.id not in (111) and accounts.clearing_code in ('LIVE')
    inner join
        dxcore.dxcore.principals as principals on accounts.owner_id = principals.id
    inner join
        dxcore.dxcore.instruments account_instrument on accounts.currency_id = account_instrument.id
    inner join
        dxcore.dxcore.activity_legs as activity_legs on activities.id = activity_legs.activity_id
    inner join
        dxcore.dxcore.instruments order_instrument on activity_legs.instrument_id = order_instrument.id
    inner join
        dxcore.dxcore.orders as orders on activities.order_id = orders.id and (orders.status is null or orders.status = 'COMPLETED')
    inner join
        dxcore.dxcore.order_legs as order_legs on orders.id = order_legs.order_id and activity_legs.leg_type = 'POS_ADJUST'
    left join
        dxcore.dxcore.orders as opening_order on (order_legs.position_code = opening_order.order_chain_id::text and opening_order.status = 'COMPLETED')
    left join
        (SELECT orders.id, coalesce(orders.extensions::json->0->'val'->'hedgingOrderId', 
            orders.extensions::json->1->'val'->'hedgingOrderId',
            orders.extensions::json->2->'val'->'hedgingOrderId',
            orders.extensions::json->3->'val'->'hedgingOrderId',  '0')::text::decimal as hedge_id FROM dxcore.dxcore.orders) hedge_order
        on orders.id = hedge_order.id
    left join
        dxcore.dxcore.order_legs hedge on hedge.order_id = hedge_order.hedge_id
    where
      activities.transaction_time >= '2023-03-15' and activities.transaction_time <= '{date_to}' 
    and
        activities.activity_type = 'TRADE'
    order by
        activities.transaction_time asc"""

with engine.connect() as conn:
    df_orders = pd.DataFrame(conn.execute(text(req_orders)))   
df_orders['transaction_time'] = pd.to_datetime(df_orders['transaction_time'])
df_orders['pnl'] = df_orders['pnl_1'] + df_orders['pnl_2'] + df_orders['pnl_3']
df_orders['symbols'] = df_orders['order_symbol'].str.partition('/')[2]
df_orders
    

,login,account_id,order_symbol,symbol,pair_type,order_id,created_time,parameters,position_code,quantity,transaction_time,price,pnl_3,pnl_1,pnl_2,markup,position_effect,pnl,symbols
0,up632596993,BTC_1_up632596993,ATOM/USD,USD,Cryptocurrencies,827018,2023-03-15 00:12:49.604,"{""SLEEP_TIME"":""200"",""MRKT_ACCEPT_PRICE"":""12.95...",800418,17.0000000000,2023-03-15,12.9545000000,0,13.89,0,0,CLOSING,13.89,USD
1,up632596993,BTC_1_up632596993,XLM/USD,USD,Cryptocurrencies,845885,2023-03-15 00:21:45.161,"{""SLEEP_TIME"":""200"",""MRKT_ACCEPT_PRICE"":""0.087...",804747,250.0000000000,2023-03-15,0.0879000000,0,0.4,0,0,CLOSING,0.4,USD
2,up632597019,USDT_2_up632597019,BTC/USD,USD,Cryptocurrencies,826203,2023-03-14 22:26:02.028,"{""MRKT_ACCEPT_PRICE"":""24635.3"",""ORDER_EXEC_STR...",826166,0.0100000000,2023-03-15,24561.7000000000,0,-0.05,0,0,CLOSING,-0.05,USD
3,up632595287,USDT_1_up632595287,ETH/USD,USD,Cryptocurrencies,859509,2023-03-15 00:31:32.369,"{""SPREAD_MARKUP_PARAMETERS"":""0.0|0.0|1.0pips|1...",859509,1.2000000000,2023-03-15,1695.5100000000,0,0,0,1.20000,OPENING,0,USD
4,up632595287,USDT_1_up632595287,BTC/USD,USD,Cryptocurrencies,859523,2023-03-15 00:31:51.567,"{""SPREAD_MARKUP_PARAMETERS"":""0.0|0.0|5.0pips|1...",859523,0.2000000000,2023-03-15,24552.4000000000,0,0,0,1.00000,OPENING,0,USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43398,up632597214,USDT_1_up632597214,MATIC/USD,USD,Cryptocurrencies,5523413,2023-07-20 21:46:12.237,"{""SLEEP_TIME"":""200"",""MRKT_ACCEPT_PRICE"":""0.769...",5523245,1000.0000000000,2023-07-20,0.7699800000,-10.26,0,0,0,CLOSING,-10.26,USD
43399,up632608332,USDT_2_up632608332,BTC/USD,USD,Cryptocurrencies,5523426,2023-07-20 21:50:39.687,"{""SLEEP_TIME"":""200"",""MRKT_ACCEPT_PRICE"":""29885...",5520774,0.1100000000,2023-07-20,29885.9000000000,45.17,0,0,0,CLOSING,45.17,USD
43400,up632608332,USDT_2_up632608332,BTC/USD,USD,Cryptocurrencies,5523434,2023-07-20 21:54:42.050,"{""SLEEP_TIME"":""200"",""MRKT_ACCEPT_PRICE"":""29895...",5523434,0.1100000000,2023-07-20,29896.9000000000,0,0,0,0,OPENING,0,USD
43401,up632595731,USDT_2_up632595731,LTC/USD,USD,Cryptocurrencies,5523200,2023-07-20 19:33:46.467,"{""MRKT_ACCEPT_PRICE"":""92.01"",""ORDER_EXEC_STRAT...",4756908,0.5000000000,2023-07-20,92.2000000000,-6.03,0,0,0,CLOSING,-6.03,USD


In [571]:
req_market_data = f"""
        select bid_time::TIMESTAMP::DATE as transaction_time, REPLACE(REPLACE(event_symbol, 'USD/', ''), '/USD', '') as symbols, --
        case
            when event_symbol in ('USD/JPY', 'USD/CNH', 'USD/MXN', 'USD/CAD', 'USD/CHF') then (1/bid_price)
            when event_symbol in ('BTC/USD', 'ETH/USD', 'EUR/USD', 'GBP/USD', 'AUD/USD', 'NZD/USD') then bid_price
        end as bid_price
    from dxcore.dxcore.quotes_history qh 
    where event_symbol in ('BTC/USD', 'ETH/USD', 'EUR/USD', 'GBP/USD', 'USD/JPY', 'USD/CNH', 
    						'USD/MXN', 'AUD/USD', 'USD/CAD', 'NZD/USD', 'USD/CHF')
    and bid_time >= '2018-12-20'
    --and bid_time::TIMESTAMP::DATE """

with engine.connect() as conn:
    df_market_data = pd.DataFrame(conn.execute(text(req_market_data))).drop_duplicates(subset=['transaction_time', 'symbols'])

df_market_data['transaction_time'] = pd.to_datetime(df_market_data['transaction_time']) 
df_market_data

,transaction_time,symbols,bid_price
0,2022-09-28,GBP,1.0833200000
1,2022-09-28,CAD,0.73379415606334111155
2,2022-09-28,JPY,0.00692271481184061141
3,2022-09-28,AUD,0.6504600000
4,2022-09-28,EUR,0.9709600000
...,...,...,...
15963,2023-07-20,CAD,0.75904785037648773379
15964,2023-07-20,BTC,29808.3000000000
15965,2023-07-20,MXN,0.05923030222261709090
15966,2023-07-20,ETH,1891.5400000000


In [572]:
df_date_symbol = df_orders.merge(df_market_data, how='left', on=['transaction_time', 'symbols'])
df_date_symbol['bid_price'] = df_date_symbol['bid_price'].fillna(1)
df_date_symbol['pnl'] = (df_date_symbol['pnl']) * df_date_symbol['bid_price']
df_date_symbol = df_date_symbol[['account_id','login', 'order_symbol', 'symbols', 'transaction_time', 'price', 'pnl', 'quantity', 'bid_price']]
df_date_symbol['volume'] = df_date_symbol['price'].astype(float) * df_date_symbol['quantity'].astype(float) * df_date_symbol['bid_price'].astype(float)
df_date_symbol['transaction_time'] = pd.to_datetime(df_date_symbol['transaction_time'])
df_date_symbol = df_date_symbol.groupby(['login']).agg({'volume': ['sum', 'mean']}).reset_index()
df_date_symbol.columns = ['login', 'volume', 'mean_volume']
df_date_symbol

,login,volume,mean_volume
0,up632542457,"9,622.02",534.56
1,up632542695,"8,790,902.62","19,069.20"
2,up632542696,"1,355.69",677.85
3,up632561969,"10,574.19","1,762.37"
4,up632574649,"48,092.16",961.84
...,...,...,...
268,up632611456,"1,251.59",208.60
269,up632611458,"5,815.66","1,453.91"
270,up632611462,"1,987.84",110.44
271,up632611463,"1,410.63",201.52


In [573]:
partners

,partner_id,partner,partner_description
0,17,17 Masha test program,Тестовая партнерка Маши. Используется для тест...
1,27,27 tiktok post CryptoMalaysia 04/23,Тестовый пост в тикток https://vt.tiktok.com/Z...
2,29,29 Google ad 05/23,Тестовая реклама в Google
3,30,30 tm Crypto Portal 04/23,Промо через телеграм канал Crypto Portal https...
4,35,35 Meta MP,"Партнер Михаил, льет трафик с Мета (facebook, ..."
5,21,21 Content@mybro,Инфлюенсер ТГ
6,41,41 TGpartner,Инфлюенсер ТГ (crypto miami)
7,42,12312345bk,Инфлюенсер ТГ (crypto mountains)
8,43,btcindustry,Инфлюенсер в тг (bitcoin industry)
9,47,crypto,Инфлюенсер ТГ (crypto miami)


In [574]:
partners = pd.read_csv('/Users/Taisia1/Desktop/octacode/partner_offer/partners.csv')
df = df.merge(partners, how='left', on='partner_id').merge(offers, how='left', on='offer_id').merge(df_date_symbol, how='left', on='login')
df.head()

,login,created_time,id,clearing_code,account_code,symbols,user_id,partner_id,offer_id,program_type,deposit_sum,deposit_mean,deposit_count,partner,partner_description,offer,offer_description,volume,mean_volume
0,up632605464,2023-03-15 07:34:13.505,94840,LIVE,USDT_1_up632605464,USDT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,335.13,167.57
1,up632605467,2023-03-15 07:59:57.309,94842,LIVE,BTC_1_up632605467,BTC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,up632605465,2023-03-15 07:59:57.447,94843,LIVE,BTC_1_up632605465,BTC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,up632605470,2023-03-15 07:59:58.116,94844,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,up632605466,2023-03-15 07:59:58.220,94845,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


нужно только вместо номера партнера его название, регистрации, демки, лайвы, депозиты и еще добавить торговые обороты и это полный ресеч трафика получится

In [575]:
df['LIVE'] = df['clearing_code']
df.loc[df['LIVE'] != 'LIVE', 'LIVE'] = None

df['DEMO'] = df['clearing_code']
df.loc[df['DEMO'] != 'DEMO', 'DEMO'] = None

In [577]:
result_partner = df.groupby(['partner_description', 'offer_description']).agg({'login': 'count', 'deposit_sum': 'sum', 'volume': 'sum', 'LIVE': 'count', 'DEMO': 'count'})
result_partner = result_partner.reset_index()
result_partner.columns = ['partner_description', 'offer_description', 'registrations', 'deposit_sum', 'volume', 'LIVE', 'DEMO']
result_partner

,partner_description,offer_description,registrations,deposit_sum,volume,LIVE,DEMO
0,Инфлюенсер ТГ,Кросс-промо Bitcoin Industry,496,0.00,0.00,83,2
1,Инфлюенсер ТГ (crypto miami),Кросс-промо Bitcoin Industry,343,0.00,0.00,80,0
2,Инфлюенсер ТГ (crypto mountains),Кросс-промо Bitcoin Industry,127,0.00,0.00,25,0
3,Инфлюенсер в тг (bitcoin industry),Кросс-промо Bitcoin Industry,176,0.00,0.00,34,0
4,"Партнер Михаил, льет трафик с Мета (facebook, ...","CPL программа для Мета (fb, ig)",4,0.00,0.00,2,0
5,"Партнер Михаил, льет трафик с Мета (facebook, ...",Основная CPA программа.,13,0.00,0.00,11,1
6,Промо через телеграм канал Crypto Portal https...,Основная CPA программа.,437,0.00,0.00,21,0
7,Тестовая партнерка Маши. Используется для тест...,Кросс-промо Bitcoin Industry,6,223.35,"6,855.02",3,2
8,Тестовая партнерка Маши. Используется для тест...,Основная CPA программа.,13,426.02,"35,174.73",8,3
9,Тестовая партнерка Маши. Используется для тест...,Основная revshare программа.,18,203.15,"37,220.40",14,3


In [578]:
key = "/Users/Taisia1/Desktop/octacode/deposite/creds.json"
client = pygsheets.authorize(service_file=key)
sh = client.open("order_statistics")
wks = sh.worksheet_by_title('statistics_partner')
wks.clear()
wks.set_dataframe(
    result_partner, (1, 1),
    copy_index=False, header=True)

In [579]:
# result_offer = df.groupby(['offer_id']).agg({'login': 'count', 'deposit_sum': 'sum'})
# result_offer = result_offer.reset_index()
# result_offer

In [580]:
# import matplotlib.pyplot as plt
# plt.figure(figsize=(10, 5))
# plt.xlabel("offer_id")
# plt.ylabel("partner, deposit_sum")
# plt.bar(result_offer['offer_id'], result_offer['login'])
# plt.plot(result_offer['offer_id'], result_offer['deposit_sum'], color='red')
# plt.show()

In [581]:
# import matplotlib.pyplot as plt
# plt.figure(figsize=(10, 5))
# plt.xlabel("partner_id")
# plt.ylabel("partner, deposit_sum")
# plt.bar(result_partner['partner_id'], result_partner['login'])
# plt.plot(result_partner['partner_id'], result_partner['deposit_sum'], color='red')
# plt.show()

In [582]:
result.to_csv('statistics_partner_offer.csv')